In [1]:
import torch
import numpy as np
import pandas as pd
import matchzoo as mz
print('matchzoo version', mz.__version__)

matchzoo version 1.2


In [2]:
ranking_task = mz.tasks.Ranking(losses=mz.losses.RankHingeLoss())
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]
print("`ranking_task` initialized with metrics", ranking_task.metrics)

`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]


In [3]:
print('data loading ...')
train_pack_raw = mz.datasets.wiki_qa.load_data('train', task=ranking_task)
dev_pack_raw = mz.datasets.wiki_qa.load_data('dev', task=ranking_task, filtered=True)
test_pack_raw = mz.datasets.wiki_qa.load_data('test', task=ranking_task, filtered=True)
print('data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`')

data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [4]:
preprocessor = mz.models.ConvKNRM.get_default_preprocessor()

In [5]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 6309.42it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 3907.59it/s]
Building Vocabulary from a datapack.: 100%|██████████| 418401/418401 [00:00<00:00, 2641216.73it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 7501.49it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 3884.55it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 485798.74it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 7809.55it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<00

In [6]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7f119e937a58>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f11a60f3518>,
 'vocab_size': 30058,
 'embedding_input_dim': 30058}

In [7]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [8]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=5,
    num_neg=1,
    batch_size=20,
    resample=True,
    sort=False
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed,
    batch_size=20
)

In [9]:
padding_callback = mz.models.ConvKNRM.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    stage='train',
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    stage='dev',
    callback=padding_callback
)

In [10]:
model = mz.models.ConvKNRM()

model.params['task'] = ranking_task
model.params['embedding'] = embedding_matrix
model.params['filters'] = 128 
model.params['conv_activation_func'] = 'tanh' 
model.params['max_ngram'] = 3
model.params['use_crossmatch'] = True 
model.params['kernel_num'] = 11
model.params['sigma'] = 0.1
model.params['exact_sigma'] = 0.001

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

ConvKNRM(
  (embedding): Embedding(30058, 300, padding_idx=0)
  (q_convs): ModuleList(
    (0): Sequential(
      (0): ConstantPad1d(padding=(0, 0), value=0)
      (1): Conv1d(300, 128, kernel_size=(1,), stride=(1,))
      (2): Tanh()
    )
    (1): Sequential(
      (0): ConstantPad1d(padding=(0, 1), value=0)
      (1): Conv1d(300, 128, kernel_size=(2,), stride=(1,))
      (2): Tanh()
    )
    (2): Sequential(
      (0): ConstantPad1d(padding=(0, 2), value=0)
      (1): Conv1d(300, 128, kernel_size=(3,), stride=(1,))
      (2): Tanh()
    )
  )
  (d_convs): ModuleList(
    (0): Sequential(
      (0): ConstantPad1d(padding=(0, 0), value=0)
      (1): Conv1d(300, 128, kernel_size=(1,), stride=(1,))
      (2): Tanh()
    )
    (1): Sequential(
      (0): ConstantPad1d(padding=(0, 1), value=0)
      (1): Conv1d(300, 128, kernel_size=(2,), stride=(1,))
      (2): Tanh()
    )
    (2): Sequential(
      (0): ConstantPad1d(padding=(0, 2), value=0)
      (1): Conv1d(300, 128, kernel_size=(3,

In [11]:
optimizer = torch.optim.Adadelta(model.parameters())
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3)

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10,
    scheduler=scheduler,
    clip_norm=10
)

In [12]:
trainer.run()

[Iter-255 Loss-0.450]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.38 - normalized_discounted_cumulative_gain@5(0.0): 0.4566 - mean_average_precision(0.0): 0.4214



[Iter-510 Loss-0.112]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3711 - normalized_discounted_cumulative_gain@5(0.0): 0.4585 - mean_average_precision(0.0): 0.4154



[Iter-765 Loss-0.060]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.321 - normalized_discounted_cumulative_gain@5(0.0): 0.4131 - mean_average_precision(0.0): 0.372



[Iter-1020 Loss-0.021]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3341 - normalized_discounted_cumulative_gain@5(0.0): 0.4198 - mean_average_precision(0.0): 0.3853



[Iter-1275 Loss-0.012]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3084 - normalized_discounted_cumulative_gain@5(0.0): 0.39 - mean_average_precision(0.0): 0.3602



[Iter-1530 Loss-0.009]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.354 - normalized_discounted_cumulative_gain@5(0.0): 0.4258 - mean_average_precision(0.0): 0.3962



[Iter-1785 Loss-0.006]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.359 - normalized_discounted_cumulative_gain@5(0.0): 0.4242 - mean_average_precision(0.0): 0.3919



[Iter-2040 Loss-0.006]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3645 - normalized_discounted_cumulative_gain@5(0.0): 0.4326 - mean_average_precision(0.0): 0.3939



[Iter-2295 Loss-0.004]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3341 - normalized_discounted_cumulative_gain@5(0.0): 0.43 - mean_average_precision(0.0): 0.386



[Iter-2550 Loss-0.005]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3272 - normalized_discounted_cumulative_gain@5(0.0): 0.4129 - mean_average_precision(0.0): 0.3799

Cost time: 567.8198726177216s
